In [1]:
import numpy as np
import pandas as pd
import time
import re
from unidecode import unidecode

In [2]:
import tls_client
from bs4 import BeautifulSoup

In [3]:
requests = tls_client.Session(
    client_identifier="chrome112",
)

In [4]:
players = pd.read_csv('player_index.csv')
player_id = players['Player-additional'].values.tolist()
player_name = players['Player'].values.tolist()
league_num =len(player_id)
i = 0
for name in player_name:
    player_name[i] = name.replace(" ", "_")
    i = i+1
print(player_name[0:10])
print(len(player_name))

['Jayson_Tatum', 'Joel_Embiid', 'Shai_Gilgeous-Alexander', 'Luka_Dončić', 'Julius_Randle', 'Damian_Lillard', 'Anthony_Edwards', 'Giannis_Antetokounmpo', 'Trae_Young', 'Donovan_Mitchell']
520


In [5]:
base = 'https://www.basketball-reference.com/players/c/bealbr01/gamelog/'
# NBA season we will be analyzing
year = '2023'
url = base+year
print(url)
# this is the HTML from the given URL
response1 = requests.get(url)
soup = BeautifulSoup(response1.content)

https://www.basketball-reference.com/players/c/bealbr01/gamelog/2023


In [6]:
def get_headers(soup):
    i = 0
    while i < 40:
        headers = [th.getText() for th in soup.findAll('tr', limit=40)[i].findAll('th')]
        i = i +1
        if headers:
            idx = i
            i = 41
    return headers[1:]
headers = get_headers(soup)
headers

['G',
 'Date',
 'Age',
 'Tm',
 '\xa0',
 'Opp',
 '\xa0',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'GmSc',
 '+/-']

In [7]:
# avoid the first header row
def get_stats_df(soup,headers,player):

    rows = soup.findAll('tr')
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
    player_stats = [x for x in player_stats if len(x) > 4]
    stats = pd.DataFrame(player_stats, columns = headers)
    stats.insert(0, 'Player', player)
    stats.index = range(len(stats))
    return stats
get_stats_df(soup,headers,'Bradley Beal')

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Bradley Beal,1,2022-10-19,29-113,WAS,@,IND,W (+7),1,37:31,...,3,5,6,0,1,3,2,23,18.1,+8
1,Bradley Beal,2,2022-10-21,29-115,WAS,,CHI,W (+2),1,36:39,...,1,1,8,0,2,5,2,19,14.3,+4
2,Bradley Beal,3,2022-10-23,29-117,WAS,@,CLE,L (-10),1,41:35,...,3,4,5,2,0,6,3,27,19.3,-5
3,Bradley Beal,4,2022-10-25,29-119,WAS,,DET,W (+21),1,21:49,...,3,4,6,0,0,2,2,13,11.3,+5
4,Bradley Beal,5,2022-10-28,29-122,WAS,,IND,L (-10),1,37:13,...,5,7,4,3,1,2,3,31,29.0,-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Bradley Beal,,2023-04-02,29-278,WAS,@,NYK,L (-9),Inactive,None,...,None,None,None,None,None,None,None,None,None,None
78,Bradley Beal,,2023-04-04,29-280,WAS,,MIL,L (-12),Inactive,None,...,None,None,None,None,None,None,None,None,None,None
79,Bradley Beal,,2023-04-05,29-281,WAS,@,ATL,L (-18),Inactive,None,...,None,None,None,None,None,None,None,None,None,None
80,Bradley Beal,,2023-04-07,29-283,WAS,,MIA,W (+6),Inactive,None,...,None,None,None,None,None,None,None,None,None,None


In [48]:
def get_stats(num):
    for person in range(num):
        # NBA season we will be analyzing
        year = '2023'
        player_abbr = player_id[person]
        person = person
        base = f'https://www.basketball-reference.com/players/c/{player_abbr}/gamelog/'
        # URL page we will scraping (see image above)
        url = base+year
        print(url)
        #time.sleep(2.95)
        # this is the HTML from the given URL
        response1 = requests.get(url)
        soup = BeautifulSoup(response1.content)
        
        headers = get_headers(soup)
        stats = get_stats_df(soup,headers,player_name[person])
        if person == 0:
            data_core = stats
        else:
            data_core = pd.concat([data_core,stats])
        print(person,player_name[person])
    return data_core


In [49]:
test = get_stats(3)

https://www.basketball-reference.com/players/c/tatumja01/gamelog/2023
0 Jayson_Tatum
https://www.basketball-reference.com/players/c/embiijo01/gamelog/2023
1 Joel_Embiid
https://www.basketball-reference.com/players/c/gilgesh01/gamelog/2023
2 Shai_Gilgeous-Alexander


In [50]:
with pd.option_context('display.max_rows', None,
                       'display.precision', 3,
                       ):
    display(test.sample(5))

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
17,Jayson_Tatum,18,2022-11-23,24-265,BOS,,DAL,W (+13),1,36:49,...,10,13,5,0,2,2,1,37,31.4,+14
10,Jayson_Tatum,11,2022-11-09,24-251,BOS,,DET,W (+16),1,31:35,...,1,1,5,1,1,1,4,31,23.5,+16
55,Shai_Gilgeous-Alexander,52,2023-02-13,24-216,OKC,,NOP,L (-3),1,38:58,...,7,10,5,1,0,6,4,24,13.2,+9
69,Shai_Gilgeous-Alexander,59,2023-03-16,24-247,OKC,@,TOR,L (-17),1,34:52,...,1,3,3,2,0,3,1,29,22.5,-17
61,Joel_Embiid,49,2023-03-02,28-351,PHI,@,DAL,L (-7),1,35:44,...,4,8,1,2,0,3,5,35,25.6,-16


In [51]:
test['Player'].unique()

array(['Jayson_Tatum', 'Joel_Embiid', 'Shai_Gilgeous-Alexander'],
      dtype=object)

In [52]:
save = test.copy(deep=True)
save.tail()

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
77,Shai_Gilgeous-Alexander,65,2023-03-31,24-262,OKC,@,IND,L (-4),1,38:14,...,9,9,4,1,1,3,6,39,29.8,
78,Shai_Gilgeous-Alexander,66,2023-04-02,24-264,OKC,,PHO,L (-10),1,37:32,...,3,3,5,2,0,1,4,39,31.8,-6
79,Shai_Gilgeous-Alexander,67,2023-04-04,24-266,OKC,@,GSW,L (-11),1,33:41,...,4,5,7,0,0,3,5,32,25.9,-9
80,Shai_Gilgeous-Alexander,68,2023-04-06,24-268,OKC,@,UTA,W (+16),1,37:43,...,5,7,7,1,0,1,3,22,16.9,+7
81,Shai_Gilgeous-Alexander,,2023-04-09,24-271,OKC,,MEM,W (+15),Inactive,None,...,None,None,None,None,None,None,None,None,None,None


In [53]:
test['Date'] = pd.to_datetime(test['Date'])
data = test
for i in range(len(data)):
    data.iloc[i,0] = unidecode(data.iloc[i,0]).replace('_',' ')
    
    

    #remove special characters

In [54]:
data = data.reset_index(drop=True)

In [55]:
data.iloc[63]

Player           Jayson Tatum
G                          60
Date      2023-03-03 00:00:00
Age                    25-000
Tm                        BOS
                             
Opp                       BRK
                      L (-10)
GS                          1
MP                      38:20
FG                         10
FGA                        23
FG%                      .435
3P                          0
3PA                         7
3P%                      .000
FT                          2
FTA                         2
FT%                     1.000
ORB                         1
DRB                        12
TRB                        13
AST                         5
STL                         1
BLK                         2
TOV                         4
PF                          2
PTS                        22
GmSc                     15.3
+/-                       -19
Name: 63, dtype: object

In [56]:
data['Player'][2]

'Jayson Tatum'

In [57]:
len(data['Player'].unique())

3

In [58]:
info = pd.read_csv('player_index.csv')
for i in range(len(info)):
    info.iloc[i,1] = unidecode(info.iloc[i,1]).replace('_',' ')
print(data['Player'][2])

p2pos = {player:pos for player,pos in zip(info['Player'].to_list(),info['Pos'].to_list())}
pos = []
for i in range(len(data)):
    pos.append(p2pos[data.iloc[i][0]])
pos

data['Pos'] = pos
data

Jayson Tatum


,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Jayson Tatum,1,2022-10-18,24-229,BOS,,PHI,W (+9),1,38:38,...,12,4,1,1,3,1,35,30.1,+10,PF
1,Jayson Tatum,2,2022-10-21,24-232,BOS,@,MIA,W (+7),1,36:47,...,5,4,1,2,2,1,29,22.7,-1,PF
2,Jayson Tatum,3,2022-10-22,24-233,BOS,@,ORL,W (+6),1,37:39,...,8,1,1,2,1,2,40,34.2,+6,PF
3,Jayson Tatum,4,2022-10-24,24-235,BOS,@,CHI,L (-18),1,35:26,...,8,5,0,0,1,2,26,20.7,-8,PF
4,Jayson Tatum,5,2022-10-28,24-239,BOS,,CLE,L (-9),1,42:11,...,7,4,1,1,4,4,32,24.1,-10,PF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,Shai Gilgeous-Alexander,65,2023-03-31,24-262,OKC,@,IND,L (-4),1,38:14,...,9,4,1,1,3,6,39,29.8,,PG
242,Shai Gilgeous-Alexander,66,2023-04-02,24-264,OKC,,PHO,L (-10),1,37:32,...,3,5,2,0,1,4,39,31.8,-6,PG
243,Shai Gilgeous-Alexander,67,2023-04-04,24-266,OKC,@,GSW,L (-11),1,33:41,...,5,7,0,0,3,5,32,25.9,-9,PG
244,Shai Gilgeous-Alexander,68,2023-04-06,24-268,OKC,@,UTA,W (+16),1,37:43,...,7,7,1,0,1,3,22,16.9,+7,PG


In [59]:
data.dropna(inplace=True)
minutes =  data['MP'].to_list()

for i in range(len(minutes)):
    new = minutes[i].split(':')
    res = float(new[0])+(float(new[1])/60)
    minutes[i] = res
data['MP'] = minutes

In [61]:
data.columns = ['Player', 'G', 'Date', 'Age', 'Tm', 'H/A', 'Opp', 'W/L', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'Pos']
home = data['H/A'].values
print(home[0:4])
for i in range(len(home)):
    if home[i] == '@':
        home[i] = 0
    else:
        home[i] = 1
home[0:10]


['' '@' '@' '@']


array([1, 0, 0, 0, 1, 1, 0, 1, 0, 0], dtype=object)

In [62]:
home[0:2]

array([1, 0], dtype=object)

In [65]:
data['W/L']

0       W (+9)
1       W (+7)
2       W (+6)
3      L (-18)
4       L (-9)
        ...   
238     W (+6)
241     L (-4)
242    L (-10)
243    L (-11)
244    W (+16)
Name: W/L, Length: 208, dtype: object

In [66]:
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,W (+9),1,38.633333,...,12,4,1,1,3,1,35,30.1,+10,PF
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,W (+7),1,36.783333,...,5,4,1,2,2,1,29,22.7,-1,PF
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,W (+6),1,37.650000,...,8,1,1,2,1,2,40,34.2,+6,PF
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,L (-18),1,35.433333,...,8,5,0,0,1,2,26,20.7,-8,PF
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,L (-9),1,42.183333,...,7,4,1,1,4,4,32,24.1,-10,PF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,Shai Gilgeous-Alexander,64,2023-03-26,24-257,OKC,0,POR,W (+6),1,35.166667,...,2,3,4,0,3,1,31,26.0,+1,PG
241,Shai Gilgeous-Alexander,65,2023-03-31,24-262,OKC,0,IND,L (-4),1,38.233333,...,9,4,1,1,3,6,39,29.8,,PG
242,Shai Gilgeous-Alexander,66,2023-04-02,24-264,OKC,1,PHX,L (-10),1,37.533333,...,3,5,2,0,1,4,39,31.8,-6,PG
243,Shai Gilgeous-Alexander,67,2023-04-04,24-266,OKC,0,GSW,L (-11),1,33.683333,...,5,7,0,0,3,5,32,25.9,-9,PG


In [67]:
data['H/A'] = home
data = data.replace('CHO','CHA')
data = data.replace('PHO','PHX')
data= data.replace('BRK','BKN')

data['W/L'] = data['W/L'].str.extract(r"\(([-+]?\d+)\)").astype(int)
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,9,1,38.633333,...,12,4,1,1,3,1,35,30.1,+10,PF
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,7,1,36.783333,...,5,4,1,2,2,1,29,22.7,-1,PF
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,6,1,37.650000,...,8,1,1,2,1,2,40,34.2,+6,PF
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,-18,1,35.433333,...,8,5,0,0,1,2,26,20.7,-8,PF
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,-9,1,42.183333,...,7,4,1,1,4,4,32,24.1,-10,PF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,Shai Gilgeous-Alexander,64,2023-03-26,24-257,OKC,0,POR,6,1,35.166667,...,2,3,4,0,3,1,31,26.0,+1,PG
241,Shai Gilgeous-Alexander,65,2023-03-31,24-262,OKC,0,IND,-4,1,38.233333,...,9,4,1,1,3,6,39,29.8,,PG
242,Shai Gilgeous-Alexander,66,2023-04-02,24-264,OKC,1,PHX,-10,1,37.533333,...,3,5,2,0,1,4,39,31.8,-6,PG
243,Shai Gilgeous-Alexander,67,2023-04-04,24-266,OKC,0,GSW,-11,1,33.683333,...,5,7,0,0,3,5,32,25.9,-9,PG


In [68]:
data['Date'].max()

Timestamp('2023-04-07 00:00:00')

In [69]:
KM_vals = pd.read_csv('KM_vals.csv',index_col=0)

In [70]:
KM_dict = dict(zip(KM_vals.Player,KM_vals.KM))

In [71]:
data.Player.values[0][0]

'J'

In [72]:
temp = []
for p in data.Player.values:
    if p[0] in KM_dict:
        temp.append(KM_dict[p[0]])
    else:
        temp.append(15)
data['KM'] = temp

In [73]:
data[['FG%','3P%','FT%']] = data[['FG%','3P%','FT%']].replace(5.00,0)

In [74]:
data.to_csv('data.csv',index=False)

In [75]:
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos,KM
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,9,1,38.633333,...,4,1,1,3,1,35,30.1,+10,PF,15
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,7,1,36.783333,...,4,1,2,2,1,29,22.7,-1,PF,15
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,6,1,37.650000,...,1,1,2,1,2,40,34.2,+6,PF,15
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,-18,1,35.433333,...,5,0,0,1,2,26,20.7,-8,PF,15
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,-9,1,42.183333,...,4,1,1,4,4,32,24.1,-10,PF,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,Shai Gilgeous-Alexander,64,2023-03-26,24-257,OKC,0,POR,6,1,35.166667,...,3,4,0,3,1,31,26.0,+1,PG,15
241,Shai Gilgeous-Alexander,65,2023-03-31,24-262,OKC,0,IND,-4,1,38.233333,...,4,1,1,3,6,39,29.8,,PG,15
242,Shai Gilgeous-Alexander,66,2023-04-02,24-264,OKC,1,PHX,-10,1,37.533333,...,5,2,0,1,4,39,31.8,-6,PG,15
243,Shai Gilgeous-Alexander,67,2023-04-04,24-266,OKC,0,GSW,-11,1,33.683333,...,7,0,0,3,5,32,25.9,-9,PG,15


In [76]:
data.Date.max()

Timestamp('2023-04-07 00:00:00')